# Submit Job to Condor

In [ ]:
import sys
import os
import numpy as np
import CBT_firstlevel
import condor

change the following

In [2]:
# USER VARIABLES ####################
EXP_DIR = '/Volumes/adcock_lab/main/studies/CBT.01'
subject_list_fname = 'CBT_subjs.txt'
#PPI_ROI = 'prob_VTA_atlas_2mm'
#PPI_ROI = 'NAc_spheres'

job runs:

In [ ]:
# INITIALIZE CONDOR CLUSTER SUBMIT JOB #########################
myJob = condor.Job()
logDir = os.path.join(EXP_DIR, 'scripts/condor_logs')
myJob.setUniverse('vanilla')                                
myJob.setOutput(os.path.join(logDir, 'output.txt'))
myJob.setError(os.path.join(logDir, 'firstlevel_error.txt'))    
myJob.setLog(os.path.join(logDir, 'log.txt'))            
myJob._generateSubmitString()
##########################################################

# BEGIN SCRIPT ######################

def runs(subj):
    files = os.path.join(EXP_DIR, 'data', 'func', subj)
    train_runs = filter(lambda x: x.startswith("TRAIN") and x.endswith(".nii.gz"), os.listdir(files))    
    return [run.rstrip(".nii.gz") for run in train_runs]


subject_list = os.path.join(EXP_DIR, 'scripts/analysis_scripts/subject_lists', subject_list_fname)
SUBJS = np.genfromtxt(subject_list, dtype='string')


for i,subj in enumerate(SUBJS):

    ################################################################# 
    for run in runs(subj):
        
        # STANDARD MODEL
        # standard_job = CBT_firstlevel.standard_model(subj, run)
        # myJob.queue(standard_job)

        # STANDARD MODEL with feedback
        standard_job = CBT_firstlevel.standard_model_with_feedback(subj, run)
        myJob.queue(standard_job)

        # PPI MODEL
        #PPI_job = SRM_firstlevel.PPI_model(subj, run, PPI_ROI)
        #myJob.queue(PPI_job)

        # THERM MODEL
        #if run[:-1] == 'TRAIN':
        #    THERM_job = SRM_firstlevel.THERM_model(subj, run)
        #    myJob.queue(THERM_job)

        print 'added jobs for ' + subj + ' run ' + run


myJob.saveSubmitFile(os.path.join(logDir, 'firstlevelSubmit.txt'))
myJob.submit()